In [12]:
from hudes.model_first.model_first_nn import param_nn_from_sequential
from hudes.models_and_datasets.mnist import MNISTCNN, MNISTCNNFlipped
import torch

mnist_width_height = 28
mnist_cnn = MNISTCNN()

mnist_flipped_param_net = MNISTCNNFlipped()
mnist_param_net = param_nn_from_sequential(mnist_cnn.net)


device = "mps"

params = torch.hstack([p.clone().reshape(-1) for p in mnist_cnn.parameters()]).reshape(
    1, -1
)
params = params.to(device).to(torch.float16)

params = torch.vstack([params, params + 0.01, params, params - 0.01])
params = params.repeat(8, 1)

batch = torch.rand(1, 32, mnist_width_height, mnist_width_height, device=device).to(
    torch.float16
)

print(params.device, batch.device, params.shape, batch.shape)

its = 100
import time

with torch.no_grad():
    st = time.time()
    k = 0
    for idx in range(its):
        out = mnist_flipped_param_net.forward(
            params, batch.repeat(params.shape[0], 1, 1, 1)
        )
        k += out[1].mean()
    print("flipped", (time.time() - st) / its, k)

    st = time.time()
    k = 0
    for idx in range(its):
        out = mnist_param_net.forward(params, batch.repeat(params.shape[0], 1, 1, 1))
        k += out[1].mean()
    print("not flipped", (time.time() - st) / its, k)

    st = time.time()
    k = 0
    for idx in range(its):
        out = mnist_flipped_param_net.forward(
            params, batch.repeat(params.shape[0], 1, 1, 1)
        )
        k += out[1].mean()
    print("flipped", (time.time() - st) / its, k)

    st = time.time()
    k = 0
    for idx in range(its):
        out = mnist_param_net.forward(params, batch.repeat(params.shape[0], 1, 1, 1))
        k += out[1].mean()
    print("not flipped", (time.time() - st) / its, k)

mps:0 mps:0 torch.Size([32, 28938]) torch.Size([1, 32, 28, 28])
flipped 0.008519978523254394 tensor(-234.8750, device='mps:0', dtype=torch.float16)
not flipped 0.010699682235717774 tensor(-234.8750, device='mps:0', dtype=torch.float16)
flipped 0.006474952697753906 tensor(-234.8750, device='mps:0', dtype=torch.float16)
not flipped 0.010698471069335937 tensor(-234.8750, device='mps:0', dtype=torch.float16)


In [3]:
params.shape

torch.Size([3, 28938])

In [5]:
params.repeat(12, 1).shape

torch.Size([36, 28938])